In [ ]:
!pip install tensorflow

In [ ]:
!pip install scikeras

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasRegressor
from sklearn.metrics import mean_squared_error

# Load the dataset
data = pd.read_csv("Residential_15.csv")

# Replacing NaN energy values with the mean
mean_energy = data['energy_kWh'].mean()
data['energy_kWh'].fillna(value=mean_energy, inplace=True)

X = data["hour"].values.reshape(-1, 1)
Y = data["energy_kWh"].values.reshape(-1, 1)

# Normalize the data (scaling to [0, 1])
scaler_X = MinMaxScaler()
scaler_Y = MinMaxScaler()
X = scaler_X.fit_transform(X)
Y = scaler_Y.fit_transform(Y)

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Define a function to create the model based on hyperparameters
def create_lstm_model(neurons_fc1=50, neurons_fc2=50, epochs=50, learning_rate=0.001):
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(1, 1)))
    model.add(Dense(neurons_fc1, activation='relu'))  # First fully connected layer
    model.add(Dense(neurons_fc2, activation='relu'))  # Second fully connected layer
    model.add(Dense(1))
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse')
    return model

# Wrap the Keras model using KerasRegressor
lstm_regressor = KerasRegressor(build_fn=create_lstm_model, verbose=0, neurons_fc1=50, neurons_fc2=50, epochs=50, learning_rate=0.001)

# Define the parameter grid for GridSearchCV
param_grid = {
    'neurons_fc1': [8, 16, 32, 64],  # Number of neurons in the first fully connected layer
    'neurons_fc2': [8, 16, 32, 64],  # Number of neurons in the second fully connected layer
    'epochs': [50, 75, 100],       # Number of training epochs
    'learning_rate': [0.025, 0.020, 0.015, 0.010],  # Learning rate
}

# Create GridSearchCV object
grid_search = GridSearchCV(estimator=lstm_regressor, param_grid=param_grid, n_jobs=-1 , cv=3)

# Fit the grid search to find the best hyperparameters
grid_search.fit(X_train, Y_train)

# Print the best hyperparameters
print("Best hyperparameters:", grid_search.best_params_)

# Get the best model
best_lstm_model = grid_search.best_estimator_

# Predict on the test set
Y_pred = best_lstm_model.predict(X_test)

# Inverse transform the predictions to get the original scale
Y_pred = scaler_Y.inverse_transform(Y_pred)
Y_test = scaler_Y.inverse_transform(Y_test)

# Calculate mean squared error
mse = mean_squared_error(Y_test, Y_pred)
print("Mean Squared Error:", mse)


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Best hyperparameters: {'epochs': 75, 'learning_rate': 0.01, 'neurons_fc1': 64, 'neurons_fc2': 64}
Mean Squared Error: 3.2555139943076887
